In [12]:
import os

In [13]:
%pwd

'c:\\Users\\SIR\\Downloads\\Data_Science_Python\\ML\\End_to_End_Project\\Chicken_Desease_Classification_DL'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\SIR\\Downloads\\Data_Science_Python\\ML\\End_to_End_Project\\Chicken_Desease_Classification_DL'

In [20]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [21]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [23]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [24]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-01 22:56:18,663: common: yaml file: config\config.yaml loaded successfully]
[2024-08-01 22:56:18,683: common: yaml file: params.yaml loaded successfully]
[2024-08-01 22:56:18,688: common: created directory at: artifacts]
[2024-08-01 22:56:18,693: common: created directory at: artifacts/data_ingestion]
[2024-08-01 22:56:33,501: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11605269
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "df430617f2fbb3c5bff0c7cbd1181e7894cc26269eca0df38389661b23123611"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F99B:1623C2:AD0AC8:CAFF02:66ABC53C
Accept-Ranges: bytes
Date: Thu, 01 Aug 2024 17:26:22 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4743-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S